## 🔑 Part 6 연습문제 정답

In [ ]:
# 사전 준비 코드 (튜토리얼의 코드를 그대로 가져옵니다)
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.figure_factory import create_dendrogram
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage

url = 'https://raw.githubusercontent.com/databricks/LearningSparkV2/master/databricks-datasets/learning-spark-v2/people/marketing_campaign.csv'
df = pd.read_csv(url, sep='\t')
numeric_features = df.select_dtypes(include=np.number).columns.tolist()
df_numeric = df[numeric_features]
preprocessor = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
X_processed = preprocessor.fit_transform(df_numeric)
X_processed = pd.DataFrame(X_processed, columns=numeric_features)
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_processed)

### 연습문제 1 정답

In [ ]:
kmeans_5 = KMeans(n_clusters=5, random_state=42, n_init='auto')
cluster_labels_5 = kmeans_5.fit_predict(X_processed)
print(f"K=5일 때의 이너셔 값: {kmeans_5.inertia_:.2f}")

# 시각화를 위한 데이터프레임 생성
df_pca_5 = pd.DataFrame(X_pca, columns=['PCA1', 'PCA2'])
df_pca_5['Cluster'] = cluster_labels_5.astype(str)

# 군집 중심점도 동일하게 PCA 변환
centroids_pca_5 = pca.transform(kmeans_5.cluster_centers_)

fig = px.scatter(df_pca_5, x='PCA1', y='PCA2', color='Cluster', title='K-Means Clustering Result (K=5)')
fig.add_trace(go.Scatter(x=centroids_pca_5[:, 0], y=centroids_pca_5[:, 1], mode='markers', marker_symbol='x', marker_color='black', marker_size=12, name='Centroids'))
fig.show()

### 연습문제 2 정답

In [ ]:
# K=4로 다시 학습
kmeans_4 = KMeans(n_clusters=4, random_state=42, n_init='auto')
cluster_labels_4 = kmeans_4.fit_predict(X_processed)

# 원본 데이터프레임에 군집 레이블 추가
df_analysis = df.copy()
df_analysis['Cluster'] = cluster_labels_4

# 군집별 특성 평균 계산
cluster_profile = df_analysis.groupby('Cluster')[['Income', 'MntWines', 'NumWebPurchases']].mean().round(2)
print(cluster_profile)

>**Cluster 0**: 소득이 가장 낮고 와인 소비량도 적은 '저소득 고객' 그룹입니다.
>
>**Cluster 1**: 소득이 높고 와인 소비량이 많은 '프리미엄 고객' 그룹입니다.
>
>**Cluster 2**: 소득이 가장 높고 와인 소비량이 압도적으로 많은 'VIP 고객' 그룹입니다.
>
>**Cluster 3**: 소득은 중간 수준이지만 웹 구매가 가장 활발한 '온라인 쇼핑족' 그룹입니다.

### 연습문제 3 정답

In [ ]:
# 'average' 연결법으로 계층적 군집 수행
agg_cluster_avg = AgglomerativeClustering(n_clusters=4, linkage='average')
agg_labels_avg = agg_cluster_avg.fit_predict(X_processed)

# 시각화
df_pca_agg_avg = pd.DataFrame(X_pca, columns=['PCA1', 'PCA2'])
df_pca_agg_avg['Agg_Cluster_Avg'] = agg_labels_avg.astype(str)

fig = px.scatter(df_pca_agg_avg, x='PCA1', y='PCA2', color='Agg_Cluster_Avg',
                 title='Hierarchical Clustering Result (K=4, Average)',
                 labels={'Agg_Cluster_Avg': '고객 군집'})
fig.show()

> 'ward' 연결법은 군집의 크기를 비교적 균등하게 나누려는 경향이 있는 반면, 'average' 연결법은 하나의 거대한 군집과 여러 개의 작은 군집들로 나뉘는 경향을 보입니다. 
> 
> 이는 'average'가 이상치에 덜 민감하기 때문일 수 있습니다. 데이터의 특성에 따라 적절한 연결법을 선택하는 것이 중요합니다.

### 연습문제 4 정답

>덴드로그램을 보면, y축 값이 약 25~30 사이에서 가장 긴 수직선이 나타납니다. 이 선을 자르면 크게 2개의 군집으로 나뉩니다. 
>
>그 다음으로 긴 수직선은 약 15~20 사이에서 나타나며, 이 선들을 자르면 3개 또는 4개의 군집으로 나눌 수 있습니다.
>
>만약 군집을 3개로 나눈다면 (y=20 근처를 자름), 매우 뚜렷하게 구분되는 세 그룹을 얻을 수 있습니다. 
>
>군집을 5개로 나누려면 (y=10~12 근처를 자름), 기존의 큰 군집 하나를 더 세분화해야 하는데, 
>
>이때 합쳐지는 군집 간의 거리가 상대적으로 짧아 군집의 특성이 뚜렷하지 않을 수 있습니다.
>
>따라서, 이 덴드로그램만 본다면 3개 또는 4개의 군집이 비교적 합리적인 선택으로 보이며, 5개는 다소 과적합(지나치게 세분화)의 위험이 있어 보입니다.

### 연습문제 5 정답

In [ ]:
dbscan_new = DBSCAN(eps=3.0, min_samples=15)
dbscan_labels_new = dbscan_new.fit_predict(X_processed)

n_clusters_new = len(set(dbscan_labels_new)) - (1 if -1 in dbscan_labels_new else 0)
n_noise_new = list(dbscan_labels_new).count(-1)

print(f"새로운 파라미터로 찾은 군집 개수: {n_clusters_new}")
print(f"새로운 파라미터로 판단된 노이즈 개수: {n_noise_new}")

>eps가 2.5에서 3.0으로 증가하면서, 각 포인트가 더 넓은 반경을 이웃으로 간주하게 됩니다. 
>
>이로 인해 기존에 분리되었던 군집들이 하나로 합쳐져 전체 군집의 개수가 줄어들었습니다. 
>
>또한, 더 많은 포인트들이 군집에 속하게 되어 노이즈의 개수도 감소했습니다. 
>
>반면 min_samples가 10에서 15로 증가하여 핵심 포인트가 되기 더 어려워졌지만, 
>
>eps 증가 효과가 더 커서 전체적으로 더 큰 군집이 형성되었습니다.

### 연습문제 6 정답

In [ ]:
gmm_spherical = GaussianMixture(n_components=4, covariance_type='spherical', random_state=42)
gmm_labels_spherical = gmm_spherical.fit_predict(X_processed)

# PCA를 이용한 시각화
df_pca['GMM_Cluster_Spherical'] = gmm_labels_spherical.astype(str)
fig = px.scatter(df_pca, x='PCA1', y='PCA2', color='GMM_Cluster_Spherical',
                 title='GMM Clustering Result (K=4, Covariance=Spherical)',
                 labels={'GMM_Cluster_Spherical': '고객 군집'})
fig.show()


>covariance_type을 'spherical'로 설정하면, GMM은 모든 군집이 원형(구형)이라고 가정합니다. 이는 K-Means의 가정과 유사합니다. 
>
>'full' 타입이 데이터의 분포에 따라 자유로운 타원형 군집을 형성하는 것과 달리,
>
>'spherical'은 더 제한적인 형태의 군집을 만듭니다.
>
>PCA 결과에서 군집의 경계가 좀 더 원에 가까운 형태로 나타나는 것을 볼 수 있습니다.

### 연습문제 7 정답

In [ ]:
birch_none = Birch(n_clusters=None)
birch_labels_none = birch_none.fit_predict(X_processed)
n_clusters_birch_none = len(pd.unique(birch_labels_none))

print(f"n_clusters=None일 때 BIRCH가 찾은 군집 개수: {n_clusters_birch_none}")


>n_clusters=None으로 설정하면, BIRCH는 CF 트리의 리프 노드에 해당하는 마이크로 클러스터들을 그대로 군집으로 반환합니다.
>
>이 때문에 사용자가 지정한 n_clusters 값(e.g., 4)보다 훨씬 많은 수의 군집이 생성됩니다. 
>
>이는 데이터의 자연스러운 세분화된 구조를 탐색하는 데 사용될 수 있으며, 이 결과를 바탕으로 최종 군집 수를 결정하는 데 힌트를 얻을 수 있습니다.

### 연습문제 8 정답

In [ ]:
optics_20 = OPTICS(min_samples=20)
optics_labels_20 = optics_20.fit_predict(X_processed)

# 도달 가능성 플롯 시각화
reachability_20 = optics_20.reachability_[optics_20.ordering_]
fig = go.Figure()
fig.add_trace(go.Bar(x=np.arange(len(reachability_20)), y=reachability_20, marker_color='red'))
fig.update_layout(title='Reachability Plot (OPTICS, min_samples=20)', 
                  xaxis_title='Data Point Order', 
                  yaxis_title='Reachability Distance')
fig.show()

> min_samples를 10에서 20으로 늘리면, 포인트가 밀집 지역(군집)의 일부로 인정받기 위한 조건이 더 까다로워집니다. 
>
> 이로 인해 도달 가능성 플롯에서 골짜기(군집)의 깊이가 전반적으로 얕아지고, 일부 작은 군집들이 사라지거나 더 큰 군집에 흡수될 수 있습니다. 
>
> 또한, 군집으로 인정되지 못하는 포인트들이 늘어나므로 노이즈의 개수가 증가하는 경향을 보입니다.